In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import time

In [61]:
print(pd.__version__)

1.5.2


In [132]:
path = r'C:\Users\pillowking\Desktop\永豐\台指\202310'
extension = 'csv'
os.chdir(path)
result = glob.glob('*Daily*.{}'.format(extension))
result

['Daily_2023_10_02.csv',
 'Daily_2023_10_03.csv',
 'Daily_2023_10_04.csv',
 'Daily_2023_10_05.csv',
 'Daily_2023_10_06.csv',
 'Daily_2023_10_11.csv',
 'Daily_2023_10_12.csv',
 'Daily_2023_10_13.csv']

In [133]:

def deal_mtx(file):    
    df = pd.read_csv(r"C:\Users\pillowking\Desktop\永豐\台指\202310\{}".format(file),encoding='CP950',low_memory=False, dtype={'到期月份(週別)':str, '開盤集合競價':str })
    df = df.iloc[:,[0,1,2,3,4,5]]
    df.columns = df.columns.str.strip()
    df['商品代號'] = df['商品代號'].str.strip()
    df = df[df.loc[:,'商品代號']=='MTX']
    a = df['到期月份(週別)'].value_counts().index.to_list()[0]
    df = df[df.loc[:,'到期月份(週別)']==a]
    df['成交時間'] = df['成交時間'].astype(str).str.zfill(6)
    df['time'] = df['成交日期'].astype(str) + df['成交時間'].astype(str)
    df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H%M%S')
    name = str(file[-10:-4]) + '.csv'
    df.to_csv(name,encoding='CP950',index=False)

for _ in result:
    all_mtx = deal_mtx(_)


all_mtx


In [71]:
df = pd.read_csv(r"C:\Users\pillowking\Desktop\永豐\台指\202309\{}".format('Daily_2023_09_01.csv'),encoding='CP950')
df = df.iloc[:,[0,1,2,3,4,5]]
df.columns = df.columns.str.strip()
df['商品代號'] = df['商品代號'].str.strip()
df = df[df.loc[:,'商品代號']=='MTX']
a = df['到期月份(週別)'].value_counts().index.to_list()[0]
df = df[df.loc[:,'到期月份(週別)']==a]
df['成交時間'] = df['成交時間'].astype(str).str.zfill(6)
df['time'] = df['成交日期'].astype(str) + df['成交時間'].astype(str)
df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H%M%S')
df = df.rename(columns={'成交數量(B+S)':'volume', '成交價格':'price'}).drop(columns=['商品代號','成交日期', '成交時間','到期月份(週別)'])
df

,price,volume,time
75515,16617.0,156,2023-08-31 15:00:00
75516,16616.0,2,2023-08-31 15:00:00
75517,16616.0,2,2023-08-31 15:00:00
75518,16616.0,2,2023-08-31 15:00:00
75519,16616.0,2,2023-08-31 15:00:00
...,...,...,...
235267,16651.0,4,2023-09-01 13:44:59
235268,16653.0,2,2023-09-01 13:44:59
235269,16650.0,2,2023-09-01 13:44:59
235270,16653.0,2,2023-09-01 13:44:59


In [145]:
path = r'C:\Users\pillowking\Desktop\永豐\台指\202308'
all_files = os.listdir(path)
target = [f for f in all_files if f.startswith('_')]
df_ls = []
for file in target:
    df = pd.read_csv(path +'\\'+ file,encoding='CP950')
    df = df.rename(columns={'成交數量(B+S)':'volume', '成交價格':'price'}).drop(columns=['商品代號','成交日期', '成交時間','到期月份(週別)'])
    mtx_price = df[['time', 'price']]
    mtx_price = mtx_price.pivot_table(values='price', 
                                index='time', 
                                aggfunc=[max, min, 'first', 'last'])

    mtx_price.columns = mtx_price.columns.droplevel(level =1)
    mtx_price = mtx_price.reset_index()
    mtx_price['time'] = pd.to_datetime(mtx_price['time'])
    days= mtx_price['time'].dt.date.unique()

    if len(days) == 3:
        t1 = pd.date_range(start=days[0]+pd.DateOffset(hours=15), end=days[1]+pd.DateOffset(hours=4, minutes=59, seconds=59), freq="s")
        t2 = pd.date_range(start=days[2]+pd.DateOffset(hours=8, minutes=45, seconds=0), end=days[2]+pd.DateOffset(hours=13, minutes=44, seconds=59),freq='s')
        times = pd.DataFrame(t1.append(t2),columns=['time'])
    else:
        t1 = pd.date_range(start=days[0]+pd.DateOffset(hours=15), end=days[1]+pd.DateOffset(hours=4, minutes=59, seconds=59), freq="s")
        t2 = pd.date_range(start=days[1]+pd.DateOffset(hours=8, minutes=45, seconds=0), end=days[1]+pd.DateOffset(hours=13, minutes=44, seconds=59),freq='s')
        times = pd.DataFrame(t1.append(t2),columns=['time'])
    ans = pd.merge(times, mtx_price, how='left', left_on='time', right_on='time')
    ans['last'] = ans['last'].fillna(method='ffill')
    ans = ans.fillna(axis=1, method='backfill')
    df_ls.append(ans)
all_mtx = pd.concat(df_ls, ignore_index=True)
all_mtx.to_csv(r'{}_min_k.csv'.format(path[-2:]),index=False)



In [134]:
df = pd.read_csv(r'C:\Users\pillowking\Desktop\永豐\台指\202309\_09_01.csv',encoding='CP950')
df = df.rename(columns={'成交數量(B+S)':'volume', '成交價格':'price'}).drop(columns=['商品代號','成交日期', '成交時間','到期月份(週別)'])
mtx_price = df[['time', 'price']]
mtx_price = mtx_price.pivot_table(values='price', 
                            index='time', 
                            aggfunc=[max, min, 'first', 'last'])

mtx_price.columns = mtx_price.columns.droplevel(level =1)
mtx_price = mtx_price.reset_index()
mtx_price['time'] = pd.to_datetime(mtx_price['time'])
days= mtx_price['time'].dt.date.unique()

if len(days) == 3:
    t1 = pd.date_range(start=days[0]+pd.DateOffset(hours=15), end=days[1]+pd.DateOffset(hours=4, minutes=59, seconds=59), freq="s")
    t2 = pd.date_range(start=days[2]+pd.DateOffset(hours=8, minutes=45, seconds=0), end=days[2]+pd.DateOffset(hours=13, minutes=44, seconds=59),freq='s')
    times = pd.DataFrame(t1.append(t2),columns=['time'])
else:
    t1 = pd.date_range(start=days[0]+pd.DateOffset(hours=15), end=days[1]+pd.DateOffset(hours=4, minutes=59, seconds=59), freq="s")
    t2 = pd.date_range(start=days[1]+pd.DateOffset(hours=8, minutes=45, seconds=0), end=days[1]+pd.DateOffset(hours=13, minutes=44, seconds=59),freq='s')
    times = pd.DataFrame(t1.append(t2),columns=['time'])

# times = times.set_index('time')
# times = times.asfreq('s')
# times = times.reset_index(level=0)
# times = times[times.loc[:,'time'].dt.dayofweek.isin([1,2,3,4,5])]
# times['time'] = times[
#                     times['time'].dt.time.between(time(8,45,00), time(13,44,59))  
#                   | times['time'].dt.time.between(time(15,00,00), time(23,59,59)) 
#                   | times['time'].dt.time.between(time(0,00,00), time(4,59,59))]
# times = times.dropna()

ans = pd.merge(times, mtx_price, how='left', left_on='time', right_on='time')

ans['last'] = ans['last'].fillna(method='ffill')

ans = ans.fillna(axis=1, method='backfill')
ans
# all_mtx = pd.concat([ans], ignore_index=True)
# all_mtx

,time,max,min,first,last
0,2023-08-31 15:00:00,16619.0,16611.0,16617.0,16615.0
1,2023-08-31 15:00:01,16619.0,16614.0,16615.0,16616.0
2,2023-08-31 15:00:02,16618.0,16615.0,16616.0,16616.0
3,2023-08-31 15:00:03,16617.0,16616.0,16616.0,16617.0
4,2023-08-31 15:00:04,16617.0,16616.0,16616.0,16616.0
...,...,...,...,...,...
68395,2023-09-01 13:44:55,16651.0,16650.0,16651.0,16650.0
68396,2023-09-01 13:44:56,16654.0,16650.0,16652.0,16653.0
68397,2023-09-01 13:44:57,16653.0,16650.0,16651.0,16653.0
68398,2023-09-01 13:44:58,16652.0,16650.0,16652.0,16652.0


In [142]:
# weekday =  all_mtx[all_mtx.loc[:,'time'].dt.dayofweek.isin([1,2,3,4])]
# weekday = weekday[ weekday['time'].dt.time.between(time(8,45,00), time(13,44,59)) 
#                   | weekday['time'].dt.time.between(time(15,00,00), time(23,59,59)) 
#                   | weekday['time'].dt.time.between(time(0,00,00), time(4,59,59))]

# sat = all_mtx[all_mtx.loc[:,'time'].dt.dayofweek==5]
# sat = sat[sat['time'].dt.time.between(time(0,00,00), time(4,59,59))]

# mon = all_mtx[all_mtx.loc[:,'time'].dt.dayofweek==0]
# mon = mon[ mon['time'].dt.time.between(time(8,45,00), time(13,44,59)) 
#                   | mon['time'].dt.time.between(time(15,00,00), time(23,59,59)) ]

# final = weekday.append(sat)
# final.append(mon)
# final

In [11]:
path = r'C:\Users\pillowking\Desktop\永豐\1min_k'
all_files = os.listdir(path)
li = []

for filename in all_files:
    df = pd.read_csv(path+'\\'+filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame

,time,max,min,first,last
0,2023-07-31 15:00:00,17109.0,17094.0,17095.0,17109.0
1,2023-07-31 15:00:01,17110.0,17103.0,17104.0,17110.0
2,2023-07-31 15:00:02,17110.0,17107.0,17107.0,17108.0
3,2023-07-31 15:00:03,17109.0,17106.0,17108.0,17106.0
4,2023-07-31 15:00:04,17106.0,17103.0,17106.0,17104.0
...,...,...,...,...,...
2872795,2023-09-28 13:44:55,16370.0,16368.0,16369.0,16370.0
2872796,2023-09-28 13:44:56,16370.0,16368.0,16369.0,16368.0
2872797,2023-09-28 13:44:57,16370.0,16368.0,16369.0,16369.0
2872798,2023-09-28 13:44:58,16370.0,16368.0,16370.0,16370.0


In [32]:
# 改變時間頻率 但要記得dropna()
df = frame.copy()
#time freq
df['time'] = pd.to_datetime(df['time'])

#make the time the index. 
df = df.set_index("time")

# group in 5-minute chunks. 
t = df.groupby(pd.Grouper(freq='min')).agg({"first": "first", 
                                             "last": "last", 
                                             "min": "min", 
                                             "max": "max"})
t.columns = ["open", "close", "low", "high"]
# t.columns = t.columns.droplevel(level =1)
t = t.reset_index()
t = t.dropna()   #需要

backtest

In [37]:
test = frame.copy()

test['h_value'] = test['max'].rolling(1320*60).max()
test['l_value'] = test['min'].rolling(930*60).min()
test = test.dropna()
test

,time,max,min,first,last,h_value,l_value
79199,2023-08-01 17:59:59,17161.0,17161.0,17161.0,17161.0,17218.0,17083.0
79200,2023-08-01 18:00:00,17161.0,17159.0,17160.0,17161.0,17218.0,17083.0
79201,2023-08-01 18:00:01,17161.0,17159.0,17159.0,17159.0,17218.0,17083.0
79202,2023-08-01 18:00:02,17161.0,17159.0,17159.0,17161.0,17218.0,17083.0
79203,2023-08-01 18:00:03,17161.0,17161.0,17161.0,17161.0,17218.0,17083.0
...,...,...,...,...,...,...,...
2872795,2023-09-28 13:44:55,16370.0,16368.0,16369.0,16370.0,16435.0,16226.0
2872796,2023-09-28 13:44:56,16370.0,16368.0,16369.0,16368.0,16435.0,16226.0
2872797,2023-09-28 13:44:57,16370.0,16368.0,16369.0,16369.0,16435.0,16226.0
2872798,2023-09-28 13:44:58,16370.0,16368.0,16370.0,16370.0,16435.0,16226.0


In [164]:
back_test = test.copy()
# back_test = back_test[(back_test.time>='2023-09-20') & (back_test.time<='2023-09-22')]

long = 0
short = 0
value = 0
profit = 0
moving_start = 0
back_point = 5
stop_loss = 35
start_moving_profit = 41
record = []
def tolist(col):
    return back_test[col].tolist()
open = tolist('first')
close = tolist('last')
h_value = tolist('h_value')
l_value = tolist('l_value')

for _ in range(len(open)):

    if long == 0:
        profit = 0
    else:
        profit = close[_] - value


    if long == 1:   #持有中
        if profit >= 41 and moving_start == 0:
            moving_start == 1
        if profit < stop_loss:
            long = 0
            record.append(['sell', close[_]])
    
    
    


    

    if (open[_] <= h_value[_] or close[_] <= h_value[_]) and long==0:
        value = close[_]
        long = 1
        record.append(['buy',close[_]])
    

        


,time,max,min,first,last,h_value,l_value
1615500,2023-09-20 00:00:00,16567.0,16565.0,16565.0,16565.0,16847.0,16555.0
1615501,2023-09-20 00:00:01,16567.0,16565.0,16565.0,16566.0,16847.0,16555.0
1615502,2023-09-20 00:00:02,16567.0,16567.0,16567.0,16567.0,16847.0,16555.0
1615503,2023-09-20 00:00:03,16568.0,16565.0,16566.0,16568.0,16847.0,16555.0
1615504,2023-09-20 00:00:04,16568.0,16565.0,16568.0,16565.0,16847.0,16555.0
...,...,...,...,...,...,...,...
1778396,2023-09-21 23:59:56,16299.0,16299.0,16299.0,16299.0,16650.0,16235.0
1778397,2023-09-21 23:59:57,16299.0,16299.0,16299.0,16299.0,16650.0,16235.0
1778398,2023-09-21 23:59:58,16300.0,16300.0,16300.0,16300.0,16650.0,16235.0
1778399,2023-09-21 23:59:59,16300.0,16300.0,16300.0,16300.0,16650.0,16235.0
